# IndicF5 Gradio Demo

This notebook installs the required packages, loads the optimized `app.py`, and launches the Gradio interface.

In [ ]:
# Clone the repository (if not already present)
import os
if not os.path.exists('f15'):
    !git clone https://github.com/beginner4a3/f15
else:
    print('Repository already cloned.')

# Change working directory to the repo
os.chdir('f15')

In [ ]:
# Install dependencies (run once)
%pip install -q transformers==4.41.2 accelerate==0.33.0 f5-tts==0.3.4 vocos==0.1.0 gradio==4.31.0 spaces huggingface_hub
# Upgrade numpy to 2.x as some libs require it in 2025
%pip install -q "numpy>=2.0" "numba>=0.60.0"
# You may need to restart the kernel after installation

In [ ]:
# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Run the app
!python app.py